# Compute a temporal aggregation using massive operators

Import PyOphidia and connect to server instance

In [ ]:
from PyOphidia import client, cube
ophclient = client.Client(read_env=True)
cube.Cube.client = ophclient

Import multiple files with a single operator call (massive operator).

**Note: Using the asterisk (\*) in the *src_path* parameter will load all the files under the specified path (5 files).**

In [ ]:
res = ophclient.submit("oph_importnc exp_dim=lat|lon;imp_dim=time;ncores=1;measure=tasmax;src_path=[path=/data/yearly_files/*.nc];description=Import cube")

Subset each imported datacube over time by selecting the JJA (June-July-August) period.

**Note: The *cube=[level=0]* argument is used to specify the list of input cubes, those from step 0 (the importnc)**

In [ ]:
res = ophclient.submit("oph_subset subset_dims=time;subset_filter=JJA;subset_type=coord;ncores=1;cube=[level=0];description=Subset Cube")

Compute average over time for each year.

**Note: The *cube=[level=1]* argument is used to specify the list of input cubes, those from step 1 (the subset)**

In [ ]:
res = ophclient.submit("oph_reduce2 operation=avg;dim=time;cube=[level=1];description=Reduced cube")

List all the produced datacubes. 

As we can see, there are 15 datacubes in total: 5 (as the imported years) for each of the previously performed operators.

In [ ]:
res=ophclient.submit("oph_list level=2",display=True)

Extract values and plot output data on map for one of the averaged years.

In the following command, replace the pid with one of the **DATACUBE PID** values related to a **Reduced cube** shown in the previous *oph_list* output.

In [ ]:
mycube = cube.Cube(pid='http://127.0.0.1/ophidia/../..')

In [ ]:
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.util import add_cyclic_point
import numpy as np
import warnings
warnings.filterwarnings("ignore")

fig = plt.figure(figsize=(15, 6), dpi=100)

#Add Geo axes to the figure with the specified projection (PlateCarree)
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)

#Draw coastline and gridlines
ax.coastlines()

gl = ax.gridlines(crs=projection, draw_labels=True, linewidth=1, color='black', alpha=0.9, linestyle=':')
gl.xlabels_top = False
gl.ylabels_right = False

data = mycube.export_array()
lat = data['dimension'][0]['values'][ : ]
lon = data['dimension'][1]['values'][ : ]
var = data['measure'][0]['values'][ : ]
var = np.reshape(var, (len(lat), len(lon)))

#Wraparound points in longitude
var_cyclic, lon_cyclic = add_cyclic_point(var, coord=np.asarray(lon))
x, y = np.meshgrid(lon_cyclic,lat)

#Define color levels for color bar
levStep = (np.nanmax(var)-np.nanmin(var))/20
clevs = np.arange(np.nanmin(var),np.nanmax(var)+levStep,levStep)

#Set filled contour plot
cnplot = ax.contourf(x, y, var_cyclic, clevs, transform=projection,cmap=plt.cm.jet)
plt.colorbar(cnplot,ax=ax)

ax.set_aspect('auto', adjustable=None)

plt.title('Maximum Near-Surface Air Temperature (deg K)')
plt.show()

Clear your workspace before running other notebooks

In [ ]:
cube.Cube.deletecontainer(container='tasmax_day_CMCC-CESM_rcp85_r1i1p1_2096.nc',force='yes')
cube.Cube.deletecontainer(container='tasmax_day_CMCC-CESM_rcp85_r1i1p1_2097.nc',force='yes')
cube.Cube.deletecontainer(container='tasmax_day_CMCC-CESM_rcp85_r1i1p1_2098.nc',force='yes')
cube.Cube.deletecontainer(container='tasmax_day_CMCC-CESM_rcp85_r1i1p1_2099.nc',force='yes')
cube.Cube.deletecontainer(container='tasmax_day_CMCC-CESM_rcp85_r1i1p1_2100.nc',force='yes')